In [1]:
#You don't need to change anything in this block, although the modules need to be installed to run this notebook

#We import numpy to handle vectors and some math
import numpy as np

#We import pandas to create a data frame of the experiment data
#Such a table can later be used for plotting our results
import pandas as pd

# Import plotly, which is used for visualization
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [7]:
class Agent:
    def __init__(self):
        self.pos = np.random.rand(2)
        self.kind = "agent"
        self.goal = np.random.rand(2)
        self.object = None
        self.k_p = 0.8 # pickup constant
        self.k_d = 0.2 # drop constant

    # update the ant's position and handle picking up/dropping objects
    def update(self, objects):
        # check if goal is reached:
        if np.linalg.norm(self.pos - self.goal) < 0.05:
            self.goal = np.random.rand(2) # assign a new goal

        gmp = self.goal - self.pos
        v = gmp / np.linalg.norm(gmp) * 0.05
        self.pos = self.pos + v
        
        # check if object pickup
        if self.object is None:
            # find object
            set_objects = [object for object in objects if not object.picked]
            set_objects = [object for object in set_objects if np.linalg.norm(object.pos - self.pos) < 0.05]
            if len(set_objects):
                if np.random.rand() < self.k_p:
                    self.object = np.random.choice(set_objects)
                    self.object.picked = True
                    self.object.pos = self.pos
                return

        # check object drop
        if self.object is None:
            return

        # update object position
        self.object.pos = self.pos
        if np.random.rand() > self.k_d:
            return

        self.object.picked = False
        self.object = None    
            

class AntObject:
    def __init__(self):
        self.pos = np.random.rand(2)
        self.kind = np.random.choice(["Apple", "Banana", "Hammer", "Nail"])
        self.picked = False



In [3]:
agents = [Agent() for _ in range(10)]
objects = [AntObject() for _ in range(30)]
t = 0

def record():
    ao = agents + objects
    return pd.DataFrame({'t': t, 'x': [a.pos[0] for a in ao], 'y': [a.pos[1] for a in ao], 'kind': [a.kind for a in ao], 'particle_id': [i for i, _ in enumerate(ao)]})

    

In [4]:
data = []
def update():
    data.append(record())
    for agent in agents:
        agent.update(objects)

for t in range(1000):
    update()

df = pd.concat(data)

In [5]:
df

,t,x,y,kind,particle_id
0,0,0.920414,0.356980,agent,0
1,0,0.657427,0.612218,agent,1
2,0,0.942997,0.093170,agent,2
3,0,0.323723,0.953137,agent,3
4,0,0.335960,0.637724,agent,4
...,...,...,...,...,...
35,999,0.941321,0.534203,Nail,35
36,999,0.184223,0.899214,Hammer,36
37,999,0.461949,0.842607,Nail,37
38,999,0.172768,0.303586,Hammer,38


In [9]:
fig = px.scatter(df, x="x", y="y", color='kind', symbol="kind", animation_frame='t', animation_group='particle_id', opacity=0.5, width=800, height=800)
fig.update_layout(xaxis_range=(0, 1.0), yaxis_range=(0, 1.0))
fig.update_traces(marker={'size': 15})
fig.show()